## Deliverable 2. Create a Customer Travel Destinations Map.

In [20]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [21]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.42,19.23,57.99,86,74,6.78,broken clouds
1,1,Mogadishu,SO,2.04,45.34,80.31,77,65,8.34,broken clouds
2,2,Katsuura,JP,35.13,140.30,66.00,59,0,11.41,clear sky
3,3,Dikson,RU,73.51,80.55,5.68,95,100,24.72,overcast clouds
4,4,Harper,LR,4.38,-7.72,77.40,86,17,6.20,light rain


In [22]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [23]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Mogadishu,SO,2.04,45.34,80.31,77,65,8.34,broken clouds
4,4,Harper,LR,4.38,-7.72,77.40,86,17,6.20,light rain
9,9,Mahebourg,MU,-20.41,57.70,73.40,94,75,3.36,shower rain
10,10,Salalah,OM,17.02,54.09,73.40,69,0,5.82,clear sky
13,13,Gayeri,BF,12.65,0.49,70.66,23,0,9.15,clear sky
18,18,Avarua,CK,-21.21,-159.78,78.80,69,45,5.82,scattered clouds
21,21,Rikitea,PF,-23.12,-134.97,75.61,83,19,10.98,few clouds
23,23,Faanui,PF,-16.48,-151.75,80.17,72,13,4.83,few clouds
26,26,Vaini,TO,-21.20,-175.20,82.40,74,20,8.05,few clouds
31,31,Kapaa,US,22.08,-159.32,78.80,78,90,16.11,overcast clouds


In [24]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                241
City                   241
Country                240
Lat                    241
Lng                    241
Max Temp               241
Humidity               241
Cloudiness             241
Wind Speed             241
Current Description    241
dtype: int64

In [25]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

clean_df.count()

City_ID                240
City                   240
Country                240
Lat                    240
Lng                    240
Max Temp               240
Humidity               240
Cloudiness             240
Wind Speed             240
Current Description    240
dtype: int64

In [26]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mogadishu,SO,80.31,broken clouds,2.04,45.34,
4,Harper,LR,77.40,light rain,4.38,-7.72,
9,Mahebourg,MU,73.40,shower rain,-20.41,57.70,
10,Salalah,OM,73.40,clear sky,17.02,54.09,
13,Gayeri,BF,70.66,clear sky,12.65,0.49,
18,Avarua,CK,78.80,scattered clouds,-21.21,-159.78,
21,Rikitea,PF,75.61,few clouds,-23.12,-134.97,
23,Faanui,PF,80.17,few clouds,-16.48,-151.75,
26,Vaini,TO,82.40,few clouds,-21.20,-175.20,
31,Kapaa,US,78.80,overcast clouds,22.08,-159.32,


In [27]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

clean_hotel_df = hotel_df.dropna(axis=0, how='any', thresh=None, subset=["Hotel Name"], inplace=False)
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mogadishu,SO,80.31,broken clouds,2.04,45.34,Hotel Juba
4,Harper,LR,77.40,light rain,4.38,-7.72,Screensaver
9,Mahebourg,MU,73.40,shower rain,-20.41,57.70,Shandrani Beachcomber Resort & Spa
10,Salalah,OM,73.40,clear sky,17.02,54.09,HAMDAN PLAZA HOTEL
18,Avarua,CK,78.80,scattered clouds,-21.21,-159.78,Paradise Inn
21,Rikitea,PF,75.61,few clouds,-23.12,-134.97,Pension Maro'i
23,Faanui,PF,80.17,few clouds,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
26,Vaini,TO,82.40,few clouds,-21.20,-175.20,Keleti Beach Resort
31,Kapaa,US,78.80,overcast clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
40,Ambon,ID,82.00,broken clouds,-3.70,128.18,Swiss-Belhotel Ambon


In [38]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [39]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [40]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))